In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, TextStreamer
from optimum.intel import OVModelForCausalLM
import openvino as ov

model_id = 'stabilityai/japanese-stablelm-base-gamma-7b'
model_vendor, model_name = model_id.split('/')

INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, tensorflow, onnx, openvino


## モデル変換 (OpenVINOモデル生成)

In [2]:
import os
import nncf

if not os.path.exists(f'{model_name}/INT4'):
    ov_model=OVModelForCausalLM.from_pretrained(model_id, export=True, compile=False, load_in_8bit=False)
    compressed_model = nncf.compress_weights(ov_model.half()._original_model, mode=nncf.CompressWeightsMode.INT4_ASYM, group_size=128, ratio=0.8)
    os.makedirs(f'{model_name}/INT4')
    ov.save_model(compressed_model, f'{model_name}/INT4/openvino_model.xml')

## OpenVINOモデルの読み込み

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

ov_model = OVModelForCausalLM.from_pretrained(
    model_id = f'{model_name}/INT4',
    device='CPU',
    ov_config={"PERFORMANCE_HINT": "LATENCY", "NUM_STREAMS": "1", "CACHE_DIR": "./cache"},
    config=AutoConfig.from_pretrained(model_id)
)

Compiling the model to CPU ...


In [7]:
def build_prompt(user_query, inputs="", sep="\n\n### "):
    sys_msg = "以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。"
    p = sys_msg
    roles = ["指示", "応答"]
    msgs = [": \n" + user_query, ": "]
    if inputs:
        roles.insert(1, "入力")
        msgs.insert(1, ": \n" + inputs)
    for role, msg in zip(roles, msgs):
        p += sep + role + msg
    return p

In [8]:
# Infer with prompt without any additional input
user_inputs = {
    "user_query": "VR とはどのようなものですか？",
    "inputs": ""
}
prompt = build_prompt(**user_inputs)

In [9]:
print(f'** Prompt:\n{prompt}\n-------------------------')
input_tokens = tokenizer(prompt, return_tensors='pt', add_special_tokens=False)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
response = ov_model.generate(**input_tokens, 
                             pad_token_id=tokenizer.eos_token_id,
                             eos_token_id=tokenizer.eos_token_id,
                             max_new_tokens=300,
                             num_return_sequences=1,
                             temperature=1.0,
                             do_sample=True,
                             top_k=5,
                             top_p=0.90,
                             repetition_penalty=1.2,
                             streamer=streamer)

** Prompt:
以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示: 
VR とはどのようなものですか？

### 応答: 
-------------------------

Virtual Reality （バーチャル・リアリティ）とは仮想現実のことだ。コンピュータが生成した視覚的または音声情報を使用して、ユーザの感覚器官や神経系を弱める方法で人間の意識を変更することを言う。例えばゲーム内で自分自身が操作キャラクターの体になって行動出来たり、映画館で見られる3D映像なんかがその典型例だね！



In [ ]:
# !optimum-cli export openvino -m stabilityai/japanese-stablelm-base-gamma-7b --trust-remote-code --weight-format int4_asym_g64 --disable-stateful japanese-stablelm-base-gamma-7b/INT4